In [54]:
import pandas as pd

file_path = 'data/crm.xlsx'
crm = pd.read_excel(file_path,parse_dates=['DT'],index_col='DT')
crm.columns =["sample","ni","fe","co","alu","sil","phos","sul","mag","cal","chrom","man","ti","xrf"]

In [55]:
crm_features = ['ni', 'fe', 'co', 'alu', 'sil', 'phos', 'sul', 'mag']

,xrf,sample,DT,ni,fe,alu,mag
5098,4,S3,2022-12-01,NaN,NaN,NaN,NaN
5099,4,S3,2022-12-02,1.00,52.98,3.48,1.00
5100,4,S3,2022-12-03,1.00,52.96,3.53,1.05
5101,4,S3,2022-12-04,NaN,NaN,NaN,NaN
5102,4,S3,2022-12-05,1.01,53.02,3.36,0.80


In [30]:
s1 = crm_daily[crm_daily['sample']=='S1']
s1.head()

,xrf,sample,DT,ni,fe,alu,sil,mag
0,2,S1,2021-06-06,0.63,48.21,10.35,1.75,0.98
1,2,S1,2021-06-30,0.64,48.96,10.66,1.68,0.91
2,2,S1,2021-07-01,0.63,48.63,10.25,1.76,0.97
3,2,S1,2021-07-02,0.63,48.64,10.20,1.80,1.00
4,2,S1,2021-07-03,0.63,48.74,10.19,1.83,1.04


In [33]:
y=s1.ni
crm_features = ['xrf','fe', 'alu', 'sil','mag']
X=s1[crm_features]

In [34]:
X.head()

,xrf,fe,alu,sil,mag
0,2,48.21,10.35,1.75,0.98
1,2,48.96,10.66,1.68,0.91
2,2,48.63,10.25,1.76,0.97
3,2,48.64,10.20,1.80,1.00
4,2,48.74,10.19,1.83,1.04


In [35]:
from sklearn.tree import DecisionTreeRegressor

#define model. specify a number for random state to ensure some results each run
s1_model = DecisionTreeRegressor(random_state=1)

#fit model
s1_model.fit(X,y)

DecisionTreeRegressor(random_state=1)

In [36]:
print("Making predictions for the following samples:")
print(X.head())
print("The predictions are")
print(s1_model.predict(X.head()))

Making predictions for the following samples:
   xrf     fe    alu   sil   mag
0    2  48.21  10.35  1.75  0.98
1    2  48.96  10.66  1.68  0.91
2    2  48.63  10.25  1.76  0.97
3    2  48.64  10.20  1.80  1.00
4    2  48.74  10.19  1.83  1.04
The predictions are
[0.63 0.64 0.63 0.63 0.63]


In [37]:
#calculate the mean absolute error
from sklearn.metrics import mean_absolute_error

predicted_ni = s1_model.predict(X)
mean_absolute_error(y, predicted_ni)


2.463412790201715e-17

In [38]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Define model
s1_model = DecisionTreeRegressor()
# Fit model
s1_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = s1_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

0.005333333333333371


In [39]:
print("First in-sample predictions:", s1_model.predict(X.head()))
print("Actual target values for ni:", y.head().tolist())

First in-sample predictions: [0.63 0.63 0.63 0.63 0.63]
Actual target values for ni: [0.63, 0.64, 0.63, 0.63, 0.63]


In [42]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

##### There are a few alternatives for controlling the tree depth, and many allow for some routes through the tree to have greater depth than other routes. But the max_leaf_nodes argument provides a very sensible way to control ***overfitting vs underfitting***. The more leaves we allow the model to make, the more we move from the underfitting area in the above graph to the overfitting area.

##### We can use a utility function to help compare MAE scores from different values for max_leaf_nodes:

In [43]:
# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  0
Max leaf nodes: 50  		 Mean Absolute Error:  0
Max leaf nodes: 500  		 Mean Absolute Error:  0
Max leaf nodes: 5000  		 Mean Absolute Error:  0
